In [39]:
import json
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from IPython.display import display, HTML

conf = SparkConf().setAppName("HomeSensors")

sc = SparkContext.getOrCreate(conf=conf)

sqlContext = SQLContext(sc)
ss = SparkSession.builder.getOrCreate()
ss.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [40]:
data = sc.textFile("./data/sensor_data.txt")
print(data.count())
data.cache()

250


./data/sensor_data.txt MapPartitionsRDD[289] at textFile at NativeMethodAccessorImpl.java:0

In [41]:
json_data = data.map(lambda x: json.loads(x))
json_data.cache()

print(json.dumps(json_data.count()))
# print(json.dumps(json_data.collect()))

250


In [42]:
sensor_data = json_data.flatMap(lambda x: x["data"])
sensor_data.cache()

print(json.dumps(sensor_data.count()))
# print(json.dumps(sensor_data.collect()))

683


In [43]:
from pyspark.sql import SparkSession
from pyspark.sql.types import Row, TimestampType
from pyspark.sql.functions import from_unixtime

sensor_data_df = sensor_data.map(lambda l: Row(**dict(l))).toDF()
sensor_data_df = sensor_data_df.withColumn('timestamp_d', from_unixtime('timestamp').cast(TimestampType()))

display(sensor_data_df)

sqlContext.registerDataFrameAsTable(sensor_data_df, "sensorData")

device_id,measurement,timestamp,type,unit,timestamp_d
temp1,20.0,1.598922E9,temperature,celsius,2020-09-01 03:00:00
temp1,19.30954994093412,1.598922E9,temperature,celsius,2020-09-01 03:00:00
temp1,19.30954994093412,1.598922E9,temperature,celsius,2020-09-01 03:00:00
temp1,19.383623550288977,1.598922E9,temperature,celsius,2020-09-01 03:00:00
temp2,25.0,1.598922E9,temperature,celsius,2020-09-01 03:00:00
temp2,25.0,1.598922E9,temperature,celsius,2020-09-01 03:00:00
temp2,24.799840734328733,1.598922E9,temperature,celsius,2020-09-01 03:00:00
temp2,24.799840734328733,1.598922E9,temperature,celsius,2020-09-01 03:00:00
temp3,25.626331819300546,1.598922E9,temperature,celsius,2020-09-01 03:00:00
temp3,25.250067810084378,1.598922E9,temperature,celsius,2020-09-01 03:00:00


In [44]:
def remove_property(data):
    del data["data"]
    return data

device_data_df = json_data.map(remove_property).map(lambda l: Row(**dict(l))).toDF()
display(device_data_df)

sqlContext.registerDataFrameAsTable(device_data_df, "deviceData")

id,location
temp1,living_room
temp2,master_bedroom
temp3,hallway
weather1,living_room
weather2,bathroom
weather3,master_bedroom
contact1,balcony
contact2,hallway
air1,master_bedroom
air2,living_room


In [45]:
result = sqlContext.sql("""
    SELECT *
    FROM deviceData
""")


result.toPandas().head()

,id,location
0,temp1,living_room
1,temp2,master_bedroom
2,temp3,hallway
3,weather1,living_room
4,weather2,bathroom
